# 1. Libraries

In [1]:
### Computational Libraries
import numpy as np
import pandas as pd

### HTML Servlet related
import requests
import markdown2
from xml.dom.minidom import parseString


### News Summary Library
import newspaper

### NLTK
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

### String Manipulation
from collections import defaultdict
from string import punctuation
from heapq import nlargest

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#### From Markdown to HTML

from webbrowser import open_new_tab


### Local HTML to PDF

import pdfcrowd
import sys

## Date for convention

from datetime import date
import calendar

# 2. Defined Classes

## 2.1 Class 'FrequencySummarizer' for Summarizing a Text

In [2]:
class FrequencySummarizer:
  def __init__(self, min_cut=0.5, max_cut=0.95):
    """ Words that have a frequency term lower than min_cut or higer than max_cut will be ignored. """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Input: a list of sentences already tokenized.
      Output:  a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """"
    Return a list of n sentences which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)

## 2.2 Function to fetch the required URL from Google-News

*  The function searches for the news as entered in the term **term**
*  The function also returns the title of the News for the fetched URL - retured by **return_title**

In [3]:
def get_google_new_results_test( term, count ):
    results = []
    results_title = []
    results_url = []
    obj = parseString( requests.get('https://news.google.com/rss/search?q={0}&hl=en-IN&gl=IN&ceid=IN:en'.format(term)).text )
    items = obj.getElementsByTagName('item') # Get each item
    for item in items[:count]:
        t,l = '', ''
        for node in item.childNodes:
            if node.nodeName == 'title':
                t = node.childNodes[0].data
            elif node.nodeName == 'link':
                l = node.childNodes[0].data
        results.append( (t,l) )
        results_title.append(t)
        results_url.append(l)
    return results_title, results_url

# 3. Web Search

*  The Data will be collected from the following News Sources only via Google-News:
    - goal.com  [HomePage](https://www.goal.com/en)
    - skysports.com  [HomePage](https://www.skysports.com/football)
    - espn.in  [HomePage](http://www.espn.in/football/?src=com)
    - theguardian.com  [HomePage](https://www.theguardian.com/football)


*  The News window is set to past 7 days.

- This is not an exhaustive list. You can customize this list by adding/removing websites. You can also filter particluar websited that you don't want in your search list.

- News wrt **Liverpool** and **English Premier League** will be fetched.
- News wrt **Liverpool** and **Champions League** will also be fetched separately.
- Then, I will combine both the News sources to only look for unique News.

# 4. News Compilation

## 4.1 Goal.com

In [16]:
## EPL
goal_epl_search = str('liverpool AND english premier league site:goal.com when:7d')

goal_epl_url = int(3) #Maximum URL's to be fetched

goal_epl_items = get_google_new_results_test( goal_epl_search, goal_epl_url )  # variable to save title and url

## Champions league
goal_cl_search = str('liverpool AND champions league site:goal.com when:7d')

goal_cl_url = int(3) #Maximum URL's to be fetched

goal_cl_items = get_google_new_results_test( goal_cl_search, goal_cl_url )  # variable to save title and url

In [20]:
##### Listing only unique URL's

goal_urls = list(set(goal_epl_items[1]).union(goal_cl_items[1]))

## 4.2 skysports.com

In [21]:
## EPL
ss_epl_search = str('liverpool AND english premier league site:goal.com when:7d')

ss_epl_url = int(3) #Maximum URL's to be fetched

ss_epl_items = get_google_new_results_test( ss_epl_search, ss_epl_url )  # variable to save title and url

## Champions league
ss_cl_search = str('liverpool AND champions league site:goal.com when:7d')

ss_cl_url = int(3) #Maximum URL's to be fetched

ss_cl_items = get_google_new_results_test( ss_cl_search, ss_cl_url )  # variable to save title and url

In [22]:
##### Listing only unique URL's

ss_urls = list(set(ss_epl_items[1]).union(ss_cl_items[1]))

## 4.3 ESPN.IN

In [23]:
## EPL
espn_epl_search = str('liverpool AND english premier league site:espn.in when:7d')

espn_epl_url = int(3) #Maximum URL's to be fetched

espn_epl_items = get_google_new_results_test( espn_epl_search, espn_epl_url )  # variable to save title and url

## Champions league
espn_cl_search = str('liverpool AND champions league site:espn.in when:7d')

espn_cl_url = int(3) #Maximum URL's to be fetched

espn_cl_items = get_google_new_results_test( espn_cl_search, espn_cl_url )  # variable to save title and url

##### Listing only unique URL's

espn_urls = list(set(espn_epl_items[1]).union(espn_cl_items[1]))

## 4.4 theguardian.com

In [27]:
## EPL
gdn_epl_search = str('liverpool AND english premier league site:theguardian.com/football when:7d')

gdn_epl_url = int(3) #Maximum URL's to be fetched

gdn_epl_items = get_google_new_results_test( gdn_epl_search, gdn_epl_url )  # variable to save title and url

## Champions league
gdn_cl_search = str('liverpool AND champions league site:theguardian.com/football when:7d')

gdn_cl_url = int(3) #Maximum URL's to be fetched

gdn_cl_items = get_google_new_results_test( gdn_cl_search, gdn_cl_url )  # variable to save title and url

##### Listing only unique URL's

gdn_urls = list(set(gdn_epl_items[1]).union(gdn_cl_items[1]))
gdn_urls

['https://www.theguardian.com/football/2019/mar/20/manchester-united-barcelona-102-ticket-prices-subsidised',
 'https://www.theguardian.com/football/2019/mar/18/premier-league-top-four-manchester-united-tottenham-arsenal-chelsea',
 'https://www.theguardian.com/football/live/2019/mar/17/fulham-v-liverpool-premier-league-live?page=with:block-5c8e6f03e4b0eff8e0fc4d50']

### URL Manipulation Techniques

In [31]:
## Common URL's in both the sets
#common = list(set(ulrs2).intersection(urls1))
#common

## Only unique URL's
#final_urls = list(set(urls1).union(ulrs2))

## Only Different URL's
#list(set(ulrs2)^set(urls1))

**Listing all the URL's together**

In [29]:
urls = list(goal_urls + ss_urls + espn_urls + gdn_urls)

# 7. Scraping Data into Markdown File

In [32]:
## Defining the Class Object ##
fs = FrequencySummarizer()

In [33]:
### Adding Naming Convension - based on Date
today_date = date.today()
today_date

day_file_name = str(today_date.day)+'/'+today_date.strftime('%m')+'/'+str(today_date.year)   ## In case Publish Date is absent
file2 = str(today_date.day)+'_'+today_date.strftime('%m')+'_'+str(today_date.year)
file_name = file2 + '_Report.pdf'

final_file_name = str(file_name)

In [34]:
%%capture output

print('<center><h3>' + str("You'll Never Walk Alone")+'</h3></center>')

for url in urls:
    article = newspaper.Article(url)
    try:
        article.download()
        article.parse()
    except:
        continue
        
    
    
    title_url = article.title
    text_text = article.text
    
#    try:
#        publish_date = str(article.publish_date.day)+'/' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
#        publish_date = str(publish_date)
#    except AttributeError:
#        publish_date = str(day_file_name)
        
    try:
        if(int(article.publish_date.month)>10):
          
            publish_date = str(article.publish_date.day)+'/' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
            publish_date = str(publish_date)
        else:
            publish_date = str(article.publish_date.day)+'/0' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
            publish_date = str(publish_date)
    except AttributeError:
        publish_date = str(day_file_name)
    
    
        ## Summarization of a News Column ##


    print('\n')
    print('<table>')

    print('<tr>')
    print('<th>')
    
    #print('[' + str(title_url)+ ']' + '('+ str(url) + ')' )
    print('<a href = "{0} ">'.format(url)+ str(title_url) +'</a>')
    #print(Date {0}.format(publish_date))
    
    #print('</div>')

    print('</th>')
    print('</tr>')
    print('</table>')              
        #print('----------------------------------')
    print('\n')
    print('\n')
    print('<article><strong>'+format(publish_date) + '</article>')
    
    
    try:
            
        for s in fs.summarize(text_text,3):
                print('*  ',s)


        print('\n')
        #print('<center>')
        #print('\n')
        #print('[Go to HeadLines]' +'('+str(url)+')')
        #print('\n')
        #print('</center>')
            #print('---')  
    except AssertionError:
        continue

    print('\n')

    
print('<table>')

print('<tr>')
print('<th>')
#print('<center><font face="verdana" color="red">' + str('Alerts!')+'</face></center>')
print('</th>')
print('</tr>')
print('</table>')              
print('\n')



In [35]:
### Getting Text of the Output
Summary = output.stdout

In [36]:
## Direct Markdown conversion of Summary in 'test_summary'
test_summary = Markdown(data= Summary)
test_summary

<center><h3>You'll Never Walk Alone</h3></center>


<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/ole-gunnar-solskjaer-latest-wayne-rooney-only-candidate-for/1vjb992rtvgj81j3vyy1wjhyo8 ">Manchester United news: Wayne Rooney backs Ole Gunnar Solskjaer for Red Devils job</a>
</th>
</tr>
</table>




<article><strong>22/03/2019</article>
*   Solskjaer the only candidate for Manchester United job, says Rooney

The former Red Devils captain believes the Norwegian manager is the right man for the dugout at Old Trafford

Wayne Rooney believes Ole Gunnar Solskjaer is the only candidate to take the full-time manager's job at , while also stating he hopes win the Premier League over this season.
*   "He is doing a fantastic job and I'm sure he is the only candidate for Manchester United to take that job and it will be great to see him get it and bring some of the players he wants to bring in and keep moving the club forward."
*   Solskjaer has revitalised United's form and morale since taking over on a caretaker basis following Jose Mourinho's departure in December.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/gross-flattered-by-liverpool-links-as-brighton-showings/1uzr7n054y14d1hv8e4dcipcdo ">Transfer news: Pascal Gross flattered by Liverpool links as Brighton showings spark exit talk</a>
</th>
</tr>
</table>




<article><strong>19/03/2019</article>
*   He told Goal and Spox of the links to those at Anfield: “Of course it is an honour when such rumours arise, but I have not heard anything concrete.”

For now, Gross’s focus is locked on events at Brighton.
*   “Of course, the national team is a dream of mine, but I have never had contact with national coach Joachim Low,” Gross said of his ambition.
*   “The change was the right move,” said Gross.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/mane-breaks-premier-league-record-as-liverpool-hot-streak/12alut43syok81uliikiiqmosf ">Liverpool news: Sadio Mane breaks Premier League record as hot streak continues</a>
</th>
</tr>
</table>




<article><strong>17/03/2019</article>
*   Mane breaks Premier League record as Liverpool hot streak continues

The 26-year-old has scored 11 goals in as many game in all competitions to keep the Reds in contention for silverware at the end of the season

Sadio Mane has now scored the most goals in a Premier League season by a Senegalese player in the history of the competition, after notching his 17th strike of the season against on Sunday.
*   They have certainly combined consistently in the last few weeks as every one of Firmino's last seven Premier League assists have been for one of his strike partners, the Brazilian setting up Mane again in west London on Sunday.
*   Article continues below

He has also drawn level with Harry Kane, Piere-Emerick Aubameyang and his team-mate Mohamed Salah on 17 goals to put him joint second in the Premier League top scorers list.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/henderson-vows-to-win-liverpool-and-england-trophies-after/6hcyzz6lny917j9ukdzl4ztb ">International football: Jordan Henderson wants Liverpool and England trophies</a>
</th>
</tr>
</table>




<article><strong>21/03/2019</article>
*   "You always want to go on, you always want to go on and win.
*   Henderson vows to win Liverpool and England trophies after near misses

After several disappointing failures for club and country, the Reds midfielder wants to "put it right" by getting his hands on silverware

Jordan Henderson is determined to win trophies with and after falling short at crucial moments during recent years.
*   "I always feel like that – my whole career I have always wanted to show how good I am and I have always wanted to improve.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/liverpool-can-easily-win-the-premier-league-the-champions-league-/urh9y2nw5z861b8mdzic4i8qf ">Liverpool news: Reds can easily win the Premier League & the Champions League, says Berger</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   “It's great that if Salah is not scoring as much as he did last season there are other players that can score – and that's exactly what the boys need to do.”
*   I don't see why they can't win the double this season.”

Salah's strike rate of late has caused concern among fans after having had such a prolific campaign last year, but Berger is not overly concerned about the Egyptian's struggles due to squad depth and others stepping up to the plate.
*   “I think they can easily win the league and they can win the Champions League too,” the 45-year-old told Sky Sports.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/liverpool-knew-they-had-a-gem-when-hoever-marked-salah-in/1sb9prdzn87lu1vqzkbidm4tl0 ">Who is Ki-Jana Hoever? Liverpool’s young Dutch defender and how he marked Mo Salah in first training session</a>
</th>
</tr>
</table>




<article><strong>18/03/2019</article>
*   Liverpool knew they had a gem when Hoever marked Salah in first training session

The Reds have high hopes for their 17-year-old Dutchman, who on Wednesday will be named among the 50 best teenagers in world football

It didn’t take Ki-Jana Hoever long to make an impact on Jurgen Klopp.
*   Hoever , a month short of his 17th birthday, held his own against Liverpool’s star man.
*   Liverpool had seen off healthy competition to land Hoever .






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/ole-gunnar-solskjaer-latest-wayne-rooney-only-candidate-for/1vjb992rtvgj81j3vyy1wjhyo8 ">Manchester United news: Wayne Rooney backs Ole Gunnar Solskjaer for Red Devils job</a>
</th>
</tr>
</table>




<article><strong>22/03/2019</article>
*   Solskjaer the only candidate for Manchester United job, says Rooney

The former Red Devils captain believes the Norwegian manager is the right man for the dugout at Old Trafford

Wayne Rooney believes Ole Gunnar Solskjaer is the only candidate to take the full-time manager's job at , while also stating he hopes win the Premier League over this season.
*   "He is doing a fantastic job and I'm sure he is the only candidate for Manchester United to take that job and it will be great to see him get it and bring some of the players he wants to bring in and keep moving the club forward."
*   Solskjaer has revitalised United's form and morale since taking over on a caretaker basis following Jose Mourinho's departure in December.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/gross-flattered-by-liverpool-links-as-brighton-showings/1uzr7n054y14d1hv8e4dcipcdo ">Transfer news: Pascal Gross flattered by Liverpool links as Brighton showings spark exit talk</a>
</th>
</tr>
</table>




<article><strong>19/03/2019</article>
*   He told Goal and Spox of the links to those at Anfield: “Of course it is an honour when such rumours arise, but I have not heard anything concrete.”

For now, Gross’s focus is locked on events at Brighton.
*   “Of course, the national team is a dream of mine, but I have never had contact with national coach Joachim Low,” Gross said of his ambition.
*   “The change was the right move,” said Gross.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/mane-breaks-premier-league-record-as-liverpool-hot-streak/12alut43syok81uliikiiqmosf ">Liverpool news: Sadio Mane breaks Premier League record as hot streak continues</a>
</th>
</tr>
</table>




<article><strong>17/03/2019</article>
*   Mane breaks Premier League record as Liverpool hot streak continues

The 26-year-old has scored 11 goals in as many game in all competitions to keep the Reds in contention for silverware at the end of the season

Sadio Mane has now scored the most goals in a Premier League season by a Senegalese player in the history of the competition, after notching his 17th strike of the season against on Sunday.
*   They have certainly combined consistently in the last few weeks as every one of Firmino's last seven Premier League assists have been for one of his strike partners, the Brazilian setting up Mane again in west London on Sunday.
*   Article continues below

He has also drawn level with Harry Kane, Piere-Emerick Aubameyang and his team-mate Mohamed Salah on 17 goals to put him joint second in the Premier League top scorers list.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/henderson-vows-to-win-liverpool-and-england-trophies-after/6hcyzz6lny917j9ukdzl4ztb ">International football: Jordan Henderson wants Liverpool and England trophies</a>
</th>
</tr>
</table>




<article><strong>21/03/2019</article>
*   "You always want to go on, you always want to go on and win.
*   Henderson vows to win Liverpool and England trophies after near misses

After several disappointing failures for club and country, the Reds midfielder wants to "put it right" by getting his hands on silverware

Jordan Henderson is determined to win trophies with and after falling short at crucial moments during recent years.
*   "I always feel like that – my whole career I have always wanted to show how good I am and I have always wanted to improve.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/liverpool-can-easily-win-the-premier-league-the-champions-league-/urh9y2nw5z861b8mdzic4i8qf ">Liverpool news: Reds can easily win the Premier League & the Champions League, says Berger</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   “It's great that if Salah is not scoring as much as he did last season there are other players that can score – and that's exactly what the boys need to do.”
*   I don't see why they can't win the double this season.”

Salah's strike rate of late has caused concern among fans after having had such a prolific campaign last year, but Berger is not overly concerned about the Egyptian's struggles due to squad depth and others stepping up to the plate.
*   “I think they can easily win the league and they can win the Champions League too,” the 45-year-old told Sky Sports.






<table>
<tr>
<th>
<a href = "https://www.goal.com/en-us/news/liverpool-knew-they-had-a-gem-when-hoever-marked-salah-in/1sb9prdzn87lu1vqzkbidm4tl0 ">Who is Ki-Jana Hoever? Liverpool’s young Dutch defender and how he marked Mo Salah in first training session</a>
</th>
</tr>
</table>




<article><strong>18/03/2019</article>
*   Liverpool knew they had a gem when Hoever marked Salah in first training session

The Reds have high hopes for their 17-year-old Dutchman, who on Wednesday will be named among the 50 best teenagers in world football

It didn’t take Ki-Jana Hoever long to make an impact on Jurgen Klopp.
*   Hoever , a month short of his 17th birthday, held his own against Liverpool’s star man.
*   Liverpool had seen off healthy competition to land Hoever .






<table>
<tr>
<th>
<a href = "http://www.espn.in/football/england/story/3805136/kane-england-winning-nations-league-would-beat-world-cup-run ">Kane: England winning Nations League would beat World Cup run</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   "If we win the Nations League, I think it will top it," Kane said, when asked to compare the tournament with the 2018 World Cup.
*   Marc Atkins/Getty Images

Manchester City, Liverpool, Tottenham and Manchester United players are all competing against each other in the Premier League and Champions League, but any risk of factions that beset previous England regimes is low, according to Kane.
*   (2:38)

England captain Harry Kane has said winning the inaugural UEFA Nations League would surpass last year's run to the World Cup semifinals.






<table>
<tr>
<th>
<a href = "http://www.espn.in/football/english-premier-league/23/blog/post/3804094/virgil-van-dijk-is-player-of-the-year-even-if-liverpool-dont-win-the-premier-league ">Virgil van Dijk is Player of the Year even if Liverpool don't win the Premier League</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   Even if Liverpool fall short in their chase for the Premier League, they can thank Van Dijk for making them such genuine contenders.
*   That run could be ended by another Anfield man, Virgil van Dijk.
*   Liverpool were probably a little more exciting last season, but van Dijk is the main reason they are far more effective this time around, thanks to his sense of calm, his authority in defence and his leadership skills.






<table>
<tr>
<th>
<a href = "http://www.espn.in/football/english-premier-league/23/blog/post/3803672/liverpool-continue-their-high-wire-act-international-break-comes-at-right-time-for-man-united ">Liverpool continue their high-wire act, international break comes at right time for Man United</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>






<table>
<tr>
<th>
<a href = "http://www.espn.in/football/club/manchester-city/382/blog/post/3803946/man-citys-road-to-the-quadruple-is-clear-but-do-they-have-the-energy-to-complete-a-historic-season ">Man City's road to the quadruple is clear, but do they have the energy to complete a historic season?</a>
</th>
</tr>
</table>




<article><strong>23/03/2019</article>
*   Manchester United famously won the Premier League, FA Cup and Champions League in 1999, while Liverpool pulled off the league, League Cup and European Cup in 1984.
*   They also won a treble of cup competitions -- FA Cup, League Cup and UEFA Cup -- in 2001.
*   Champions League

If City had to choose between competitions over the final two months, it is the Champions League that Guardiola and City's Abu Dhabi hierarchy are most desperate to win.






<table>
<tr>
<th>
<a href = "https://www.theguardian.com/football/2019/mar/20/manchester-united-barcelona-102-ticket-prices-subsidised ">Barcelona to charge Manchester United fans £102 for ticket at Camp Nou</a>
</th>
</tr>
</table>




<article><strong>20/03/2019</article>
*   Anderlecht’s Uefa charge offers hope for fleeced Liverpool and United fans Read more

This makes it the most expensive United ticket outside of a final, with the club explaining they will use the extra revenue from the first leg on 10 April to lower the price for their fans travelling to Barcelona on 16 April, bringing down the cost of the ticket to £75.
*   Manchester United are to subsidise the £102 cost for a ticket at Barcelona for the second leg of the Champions League quarter-final and have taken a “difficult decision” to charge the same price for Barcelona supporters at Old Trafford.
*   The Belgian club raised the cost to €100 for a game in November 2017, with many Bayern fans boycotting the match despite their club subsidising the ticket price to around €70.






<table>
<tr>
<th>
<a href = "https://www.theguardian.com/football/2019/mar/18/premier-league-top-four-manchester-united-tottenham-arsenal-chelsea ">Premier League fight for top four is on – just don’t mention the b-word</a>
</th>
</tr>
</table>




<article><strong>18/03/2019</article>
*   Quick guide Premier League top-four race: remaining fixtures Show Hide The four-way sprint for the third and fourth Champions League places Tottenham

31 Mar Liverpool (A)

3 Apr Crystal Palace (H)

7 Apr Brighton (H)

13 Apr Huddersfield (H)

20 Apr Man City (A)

27 Apr West Ham (H)

4 May Bournemouth (A)

12 May Everton (H)



Arsenal

1 Apr Newcastle (H)

7 Apr Everton (A)

15 Apr Watford (A)

21 Apr Crystal Palace (H)

24 Apr Wolves (A)

29 Apr Leicester (A)

4 May Brighton (H)

12 May Burnley (A) Manchester United

30 Mar Watford (H)

2 Apr Wolves (A)

13 Apr West Ham (H)

21 Apr Everton (A)

24 Apr Man City (H)

28 Apr Chelsea (H)

4 May Huddersfield (A)

12 May Cardiff (H) Chelsea 31 Mar Cardiff (A)

3 Apr Brighton (H)

8 Apr West Ham (H)

14 Apr Liverpool (A)

22 Apr Burnley (H)

28 Apr Man Utd (A)

4 May Watford (H)

12 May Leicester (A)





With only four points separating the teams occupying third to sixth it could be anybody’s game.
*   If a Premier League team win the Champions League and finish outside the top four, and Arsenal or Chelsea win the Europa League and finish outside the top four, then only the top three will join them.
*   Tottenham, Arsenal, Manchester United and Chelsea can all still win a European trophy but, if they don’t, have just eight games each to try to secure a Champions League place

It is hard to place the exact moment when “bottle” became such an overused part of football’s lingua franca but nowadays it is hard to get through a weekend without several cases.






<table>
<tr>
<th>
<a href = "https://www.theguardian.com/football/live/2019/mar/17/fulham-v-liverpool-premier-league-live?page=with:block-5c8e6f03e4b0eff8e0fc4d50 ">Fulham 1-2 Liverpool: Premier League – as it happened</a>
</th>
</tr>
</table>




<article><strong>17/03/2019</article>
<table>
<tr>
<th>
</th>
</tr>
</table>




### Conversion of Markdown to HTML

In [37]:
html = markdown2.markdown(str(Summary), html4tags=True, tab_width=5,  )

### Cleaning the HTML File

In [39]:
### Rectify Out of Bound Paras
html = html.replace("<p>","<ul><li><p>")

html = html.replace('</p>', '</p></li></ul>')

#### Rectify Headlines Link
html = html.replace("<ul><li><p><a href","<p><a href")

html = html.replace('/a></p></li></ul>', '/a></p>')

###  Rectify <centre>
html = html.replace("<ul><li><p><center></p></li></ul>","<p><center></p>")

html = html.replace('<ul><li><p></center></p></li></ul>', '<p></center></p>')


### Rectify *

html = html.replace("*","</li><li>")

### Rectify Image

html = html.replace("<li><img ","<lily imagine ")

#html = html.replace("</h1></center></p></li></ul>","</h1></center></p>")

## Rectify Title Bullet point

html = html.replace("<ul><li><p><center><h3>","<p><center><h3>")

html = html.replace("</h3></center></p></li></ul>","</h3></center></p>")

## Open in new tab

html = html.replace("<a href", '<a rel="noopener noreferrer" target="_blank" href')

### Checking the sample Webpage

In [43]:
filename = 'File.html'

f = open(filename,'w')

wrapper = """<!DOCTYPE html>
<html lang="en">



<head>

<style>
.center {
  text-align: center;
  
}




</style>


<meta charset="ISO-8859-1" >
<div class="page center">
   <a href="https://www.liverpoolfc.com/welcome-to-liverpool-fc"><img class="auto-size__target postpone-load--fade-in widge-figure__image" data-src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" alt="" itemprop="image" src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" style="PADDING-RIGHT: 70px"></a>
</div>

<title>%s</title>

</head>

<body>
    
    %s
</body>



</html>"""

title = str('International and National Apple Market Overview')

whole = wrapper % (title, html)

#whole = wrapper % (html)

f.write(whole)

f.close()

open_new_tab(filename)

True

## Conversion to PDF

In [44]:
try:
    # create the API client instance
    client = pdfcrowd.HtmlToPdfClient('<User_Name>', '<API_Key>')

    # run the conversion and write the result to a file
    client.convertFileToFile('File.html', final_file_name )
except pdfcrowd.Error as why:
    # report the error
    sys.stderr.write('Pdfcrowd Error: {}\n'.format(why))

    # handle the exception here or rethrow and handle it at a higher level
    raise